## Setup

In [ ]:
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import pickle
from itertools import combinations

import pprint
import json
import glob
from random import random, randrange
import logging
from collections import defaultdict
import sys

import subprocess
import shlex

import scipy
from scipy import stats
from scipy.stats import pearsonr

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from datetime import datetime, timedelta
from datetime import datetime
from dateutil.parser import parse

import datetime
from collections import Counter
import itertools
import time
import csv
import pandas as pd
import pandas
from tqdm import trange, tqdm

import scipy
from scipy import stats
from scipy.stats import pearsonr

start_time = time.time()

In [ ]:
# Using the regular dictionary
f = open('author_popularity_dictionary','rb')
popularity_dict = pickle.load(f)
f.close()

com_data = pd.read_csv("PT-pruned-louvain.gexf.csv")

## Now to the popularity of those who were always active

In [ ]:
clusters = [13,48,6,49,104,57,85,80]
labels = ["Democrats","Republicans","Unorthodox","Public Health","Anti-Vaxxers","French","Indian","FreedomUK"]

In [ ]:
def get_com_nodes(com):
    com_nodes = []
    subset = com_data[com_data['modularity_class']==com]
    for i in subset['Id']:
        com_nodes.append(i)
    return com_nodes

In [ ]:
com_data

In [ ]:
def get_node_com(node):
    com = com_data[com_data['Id']==node]['modularity_class']
    val = com.values[0]
    return val

In [ ]:
get_node_com('johnjosephTX')

In [ ]:
def check_auth_time(author,pre=True):
    date_break = datetime.datetime.strptime('11/03/20',"%d/%m/%y")
    data = popularity_dict[author]
    value = False
    for entry in data:
        date = datetime.datetime.strptime(entry[0],'%y-%m-%d-%H:%M:%S')
        if pre:
            if date <= date_break:
                value=True
        else:
            if date_break <= date:
                value=True
    return value

def check_auth_continuity(author):
    date_break = datetime.datetime.strptime('11/03/20',"%d/%m/%y")
    data = popularity_dict[author]
    pre = False
    post = False
    for entry in data:
        date = datetime.datetime.strptime(entry[0],'%y-%m-%d-%H:%M:%S')
        if date <= date_break:
            pre=True
        if date_break <= date:
            post=True
    if (pre & post):
        return True
    else:
        return False

In [ ]:
def get_mean_followers(com,pre=True):
    date_break = datetime.datetime.strptime('11/03/20',"%d/%m/%y")
    com_nodes = get_com_nodes(com)
    # Consider only those who were active before and after  
    com_auth = []
    for node in com_nodes:
        if check_auth_continuity(node):
            com_auth.append(node)   
    com_norm = len(com_auth)
    com_count = float(0)
    for author in com_auth: # only those active before and after
        data = popularity_dict[author]
        auth_norm = len(data)
        auth_count = float(0)
        for entry in data:
            date = datetime.datetime.strptime(entry[0],'%y-%m-%d-%H:%M:%S')
            if pre:
                if date <= date_break:
                    auth_count += entry[1] # count followers
                    auth_count += entry[2] # count friends
            else:
                if date_break <= date:
                    auth_count += entry[1] # count followers
                    auth_count += entry[2] # count friends                    
        auth_mean = auth_count/auth_norm
        com_count += auth_mean
    com_mean = com_count/com_norm
    return com_mean

In [ ]:
recompute = False

if recompute:
    results = pd.DataFrame(columns=['Community_Name','Community_Id','Prior_Active_Users','Posterior_Active_Users',\
                                    'Continuous_Active_Users','Prior_Mean','Posterior_Mean'])
    for i in tqdm(range(len(clusters))):
        data = []
        data.append(labels[i])
        data.append(clusters[i])
        com_nodes = get_com_nodes(clusters[i])
        prior_count = 0
        posterior_count = 0
        continuity_count = 0
        for node in com_nodes:
            if check_auth_time(node,pre=True):
                prior_count += 1
            if check_auth_time(node,pre=False):
                posterior_count += 1
            if check_auth_continuity(node):
                continuity_count += 1
        data.append(prior_count)
        data.append(posterior_count)
        data.append(continuity_count)
        prior_mean = get_mean_followers(clusters[i],pre=True)
        data.append(prior_mean)
        posterior_mean = get_mean_followers(clusters[i],pre=False)
        data.append(posterior_mean)
        results.loc[len(results)] = data
    results.to_csv('prepost_followers.csv', index=False)
    results

In [ ]:
data = pandas.read_csv('prepost_followers.csv')
data = data.drop(columns=['Unnamed: 0'])
data
#data.to_csv('prepost_followers.csv', index=False)

In [1]:
import pandas
data = pandas.read_csv('prepost_followers.csv')
data

,Community_Name,Community_Id,Prior_Active_Users,Posterior_Active_Users,Continuous_Active_Users,Prior_Mean,Posterior_Mean
0,Dems,13,51319,87173,46700,8118.406542,25295.379267
1,Reps,48,33878,68566,32197,3321.962841,15161.272587
2,Disenfranch,6,22596,57201,19770,4303.805005,12573.280748
3,Health,49,25639,45190,21128,6236.866290,12090.551444
4,Antivax,104,14009,26321,11312,1457.591635,3521.396328
5,French,57,10449,27244,9056,3291.912570,11467.405191
6,Indian,85,7260,20858,6327,12996.462042,49057.459902
7,FreedomUK,80,3616,10626,3280,2657.267008,15368.555077


In [ ]:
data.rename(columns={"Community Name": "bla", "Community Id": "Community_Id"})
data

## Post-Followers Correlation with Community

For each node:

- X = [mean posterior (followers + friends),mean prior (followers + friends)]
- Y = [posterior tweet count,prior tweet count]
- Z = its community
- ID = Id
- Other statistics

Is there a correlation between X and Y?

### Setup

In [ ]:
generate_data = False

In [ ]:
if generate_data:
    PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
def get_author_data(auth):
    data = popularity_dict[auth]
    date_break = datetime.datetime.strptime('11/03/20',"%d/%m/%y")
    prior = np.array([0,0]).astype('int64')
    posterior = np.array([0,0]).astype('int64')
    for entry in data:
        date = datetime.datetime.strptime(entry[0],'%y-%m-%d-%H:%M:%S')
        if date <= date_break:
            prior[0] += entry[1] + entry[2] # count followers and friends
            prior[1] += 1
        if date_break <= date:
            posterior[0] += entry[1] + entry[2] # count followers and friends
            posterior[1] += 1
    prior_mean = prior[0]/prior[1]
    posterior_mean = posterior[0]/posterior[1]
    
    ID = auth
    C = get_node_com(auth)
    X_1 = prior_mean
    X_2 = posterior_mean
    Y_1 = prior[1]
    Y_2 = posterior[1]
    R = PT.nodes[auth]['total_retweets']
    F = PT.nodes[auth]['neighbors']
    I = PT.nodes[auth]['w_indegree']
    
    return np.array([ID,C,X_1,X_2,Y_1,Y_2,R,F,I])


In [ ]:
if generate_data:
    # Try it out
    for author in popularity_dict:
        if check_auth_continuity(author):
            winner = author
            break   
    get_author_data(winner)

In [ ]:
if generate_data:
    results = pd.DataFrame(columns =['Id','Community', 'Prior_Mean_Followers', 'Posterior_Mean_Followers', \
                                     'Prior_Tweet_Count', 'Posterior_Tweet_Count','Retweets',\
                                     'Neighbors','In_Degree'])
    
    for author in tqdm(popularity_dict):
        if check_auth_continuity(author):
            data = get_author_data(author)
            results.loc[len(results)] = data

    results.to_csv("user_features", index=False)

## Pearsons

In [ ]:
# Load and prepare the data
data = pd.read_csv("user_features")
X = data['Prior_Tweet_Count']-data['Posterior_Tweet_Count']
Y = data['Prior_Mean_Followers']-data['Posterior_Mean_Followers']
Z = data['Community']

# Try it out
pearsons1 = True        
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
if pearsons1:
    corr1 = scipy.stats.pearsonr(X,Y)
    print(corr1)
    corr2 = scipy.stats.pearsonr(Z,Y)
    print(corr2)

# Pandas is the best
pearsons2 = True

if pearsons2:
    # Pandas data frame from X and Y
    XY1 = pd.DataFrame(list(zip(X, Y)), 
                   columns =['Delta_Twit_Count', 'Delta_Mean'])
    print(XY1.corr(method='pearson', min_periods=1))
    XY2 = pd.DataFrame(list(zip(Z, Y)), 
                   columns =['Community', 'Delta_Mean'])
    print(XY2.corr(method='pearson', min_periods=1))
    

## Generate Community Aggregate Information

In [ ]:
clusters = [13,48,6,49,104,57,85,80]
labels = ["Democrats","Republicans","Unorthodox","Public Health","Anti-Vaxxers","French","Indian","FreedomUK"]

In [ ]:
def get_node_com(node):
    com = com_data[com_data['Id']==node]['modularity_class']
    val = com.values[0]
    return val

def check_auth_continuity(author):
    date_break = datetime.datetime.strptime('11/03/20',"%d/%m/%y")
    data = popularity_dict[author]
    pre = False
    post = False
    for entry in data:
        date = datetime.datetime.strptime(entry[0],'%y-%m-%d-%H:%M:%S')
        if date <= date_break:
            pre=True
        if date_break <= date:
            post=True
    if (pre & post):
        return True
    else:
        return False

In [ ]:
data

In [ ]:
nodes = ['GreatTammie','CjPolka','RMindedPatriots']
for node in nodes:
    print(data[data['Id']==node]['Retweets'])
type(nodes)

In [ ]:
data = pd.read_csv("user_features")

In [ ]:
data[data['Id']=='CjPolka']['In_Degree']

In [ ]:
com_nodes = get_com_nodes(clusters[7])
    
Retweets = float(0)
Retweeted = float(0)
for node in tqdm(com_nodes):
    Retweets += data[data['Id']==node]['Retweets']
    Retweeted += data[data['Id']==node]['In_Degree']

Retweets

In [ ]:
for node in tqdm(com_nodes):
    print(data[data['Id']==node]['Retweets'])

In [ ]:
Retweets

In [ ]:
recompute = True

In [ ]:
if recompute:
    PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
PT.nodes['Dr_Ivanoncologo']

In [ ]:
if recompute:
    results = pd.DataFrame(columns=['Community_Name','Community_Id','Retweets','Retweeted'])

    for i in tqdm(range(len(clusters))):
        row = []
        row.append(labels[i])
        row.append(clusters[i])
        com_nodes = get_com_nodes(clusters[i])
        Retweets = int(0)
        Retweeted = int(0)
        for node in tqdm(com_nodes):
            Retweets += PT.nodes[node]['total_retweets']
            Retweeted += PT.nodes[node]['w_indegree']
        row.append(int(Retweets))
        row.append(int(Retweeted))
        # Beautiful trick
        results.loc[len(results)] = row

        results.to_csv('tweet_flow.csv', index=False)

In [ ]:
data = pandas.read_csv('prepost_followers.csv')
data

In [ ]:
results

In [ ]:
frames = [data,results]
total = pd.concat(frames, keys='Community_Id', axis = 1)
total.to_csv('community_features.csv', index=False)

In [ ]:
total

In [ ]:
results = pd.DataFrame(columns=['Community_Name','Community_Id','Retweets','Retweeted'])

for i in tqdm(range(len(clusters))):
    row = []
    row.append(labels[i])
    row.append(clusters[i])
    com_nodes = get_com_nodes(clusters[i])
    Retweets = int(0)
    Retweeted = int(0)
    for node in tqdm(com_nodes):
        Retweets += PT.nodes[node]['total_retweets']
        Retweeted += PT.nodes[node]['w_indegree']
    row.append(int(Retweets))
    row.append(int(Retweeted))
    # Beautiful trick
    results.loc[len(results)] = row
    
    results

In [ ]:
results

In [ ]:
results

In [ ]:
results.to_csv('Com_ret_ret.csv', index=False)

In [ ]:
or node in com_nodes:
    Retweets += data[data['Id']==node]['Retweets']
    Retweeted += data[data['Id']==node]['In_Degree']

In [ ]:
recompute = True

In [ ]:
recompute = False

if recompute:
    results = pd.DataFrame(columns=['Community_Name','Community_Id','Prior_Active_Users','Posterior_Active_Users',\
                                    'Continuous_Active_Users','Prior_Mean','Posterior_Mean'])
    for i in tqdm(range(len(clusters))):
        data = []
        data.append(labels[i])
        data.append(clusters[i])
        com_nodes = get_com_nodes(clusters[i])
        prior_count = 0
        posterior_count = 0
        continuity_count = 0
        for node in com_nodes:
            if check_auth_time(node,pre=True):
                prior_count += 1
            if check_auth_time(node,pre=False):
                posterior_count += 1
            if check_auth_continuity(node):
                continuity_count += 1
        data.append(prior_count)
        data.append(posterior_count)
        data.append(continuity_count)
        prior_mean = get_mean_followers(clusters[i],pre=True)
        data.append(prior_mean)
        posterior_mean = get_mean_followers(clusters[i],pre=False)
        data.append(posterior_mean)
        results.loc[len(results)] = data
    results.to_csv('prepost_followers.csv', index=False)
    results

In [ ]:
results1 = pd.DataFrame(columns =['Id','Community', 'Prior_Mean_Followers', 'Posterior_Mean_Followers', \
                                 'Prior_Tweet_Count', 'Posterior_Tweet_Count','Retweets',\
                                 'Neighbors','In_Degree'])

results2 = pd.DataFrame(columns=['Community_Name','Community_Id','Prior_Active_Users','Posterior_Active_Users',\
                                    'Continuous_Active_Users','Prior_Mean','Posterior_Mean'])

In [ ]:
generate_com_data = False

In [ ]:
if generate_com_data:
    results2 = pd.DataFrame(columns=['Community_Name','Community_Id','Prior_Active_Users','Posterior_Active_Users',\
                                    'Continuous_Active_Users','Prior_Mean','Posterior_Mean','Retweets','Retweeted'])
    

In [ ]:
data

## Classifiers

In [ ]:
# Load Data
data = pd.read_csv("user_features_original_working")

feature_cols = ['Delta_Twit_Count','Community']
X = data[feature_cols]
y = data.Delta_Mean
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

type(X_train.iloc[(0,1)])

### Linear Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

In [ ]:
linear = LinearRegression()

linear.fit(X_train,y_train)
y_linear_pred=linear.predict(X_test)

# Confusion matrix
cnf_matrix = metrics.confusion_matrix(y_test, y_linear_pred)
print(cnf_matrix)

# Coefficients
print(linear.coef_)

## Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

In [ ]:
logreg = LogisticRegression()

logreg.fit(X_train,y_train)
y_logreg_pred=logreg.predict(X_test)

# Confusion matrix
cnf_matrix = metrics.confusion_matrix(y_test, y_logreg_pred)
print(cnf_matrix)

# Coefficients
print(logreg.coef_)

### Merge Data

In [ ]:
main_dict = pd.read_csv("PT-pruned-louvain.gexf.csv")

In [ ]:
main_dict

In [ ]:
main_dict.drop(columns=['Label', '0','1'])
main_dict

In [ ]:
data

In [ ]:
data = pd.read_csv("user_features")
#data.drop(columns=['Unnamed: 0'])
data.merge(main_dict, on = 'Id')
data

In [ ]:
data = pd.DataFrame(list(zip(X, Y,Z)), 
               columns =['Delta_Mean', 'Delta_Twit_Count', 'Delta_Twit_Count'])

In [ ]:
data

In [ ]:
# list of strings 
lst = ['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks'] 
  
# list of int 
lst2 = [11, 22, 33, 44, 55, 66, 77] 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(lst, lst2)), 
               columns =['Name', 'val'])

In [ ]:
copy['Delta_Mean'] = copy.apply(lambda x: get_author_XYZ(copy.['Id'][x], axis=1)
copy['Delta_Twit_Count'] = copy.apply(lambda x: get_author_XYZ(x['Id'][1], axis=1)
#copy['Delta_Twit_Count'] = copy.apply(lambda x: my_function(x['Id'][0], axis=1)
get_author_XYZ

In [ ]:
# Pandas is the best
# Pandas data frame from X and Y
XY = pd.DataFrame(list(zip(X, Y)), 
               columns =['Delta_Mean', 'Delta_Twit_Count'])
XY.corr(method='pearson', min_periods=1)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

## Add data to main Dictionary

In [ ]:
## Add this data to the main dictionary, because it is useful
com = com_data[com_data['Id']=='johnjosephTX']['modularity_class']
val = com.values[0]
val

In [ ]:
copy = pd.read_csv("PT-pruned-louvain.gexf.csv")

In [ ]:
copy

In [ ]:
for item in copy['Id']:
    print(type(item))

In [ ]:
type(get_author_XYZ(copy['Id'][2])[0])

In [ ]:
for author in copy['Id']:
    print(get_author_XYZ(copy['Id'][author])[0])
    #copy['Delta_Mean'] = get_author_XYZ(copy['Id'][author])[0]

copy
#copy['Delta_Mean'] = copy.apply(lambda x: get_author_XYZ(copy['Id'][x])[0], axis=1)

In [ ]:
copy['Delta_Mean'] = copy.apply(lambda x: get_author_XYZ(copy.['Id'][x], axis=1)
copy['Delta_Twit_Count'] = copy.apply(lambda x: get_author_XYZ(x['Id'][1], axis=1)
#copy['Delta_Twit_Count'] = copy.apply(lambda x: my_function(x['Id'][0], axis=1)
get_author_XYZ

In [ ]:
df['price'] = df['Symbol'].apply(getquotetoday)

In [ ]:
df['new_column_name'] = df.apply(lambda x: my_function(x['value_1'], x['value_2']), axis=1)


## Pearsons

In [ ]:
import scipy
from scipy import stats
from scipy.stats import pearsonr

X = []
Y = []

for author in tqdm(popularity_dict):
    if check_auth_continuity(author):
        data = get_author_XY(author)
        X.append(data[0])
        Y.append(data[1])
        
corr = scipy.stats.pearsonr(X,Y)

In [ ]:
# Pandas is the best
corr(method='pearson', min_periods=1)

In [ ]:
elapsed_time2 = elapsed_time2 - elapsed_time
print(elapsed_time2)

In [ ]:
corr

In [ ]:
elapsed_time2 = elapsed_time2 - elapsed_time
print(elapsed_time2)